https://github.com/JonasHeinzmann-AI/G2Net-Gravitational-Wave-Detection/blob/main/g2net-eda-and-modeling.ipynb

In [ ]:
!pip install efficientnet_pytorch -qq

In [ ]:
import time

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
from torch.autograd import Variable
import efficientnet_pytorch
from sklearn.model_selection import StratifiedKFold

In [ ]:

def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True


set_seed(42)

In [ ]:
class DataRetriever(torch_data.Dataset):
    def __init__(self, paths, targets):
        self.paths = paths
        self.targets = targets
        
        self.q_transform = CQT1992v2(
            sr=2048, fmin=20, fmax=1024, hop_length=32
        )
          
    def __len__(self):
        return len(self.paths)
    
    def __get_qtransform(self, x):
        image = []
        for i in range(3):
            waves = x[i] / np.max(x[i])
            waves = torch.from_numpy(waves).float()
            channel = self.q_transform(waves).squeeze().numpy()
            image.append(channel)
            
        return torch.tensor(image).float()
    
    def __getitem__(self, index):
        file_path = convert_image_id_2_path(self.paths[index])
        x = np.load(file_path)
        image = self.__get_qtransform(x)
        
        y = torch.tensor(self.targets[index], dtype=torch.float)
            
        return {"X": image, "y": y}

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = efficientnet_pytorch.EfficientNet.from_pretrained("efficientnet-b7")
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
    
    def forward(self, x):
        out = self.net(x)
        return out

In [ ]:
class LossMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0

    def update(self, val):
        self.n += 1
        # incremental update
        self.avg = val / self.n + (self.n - 1) / self.n * self.avg

        
class AccMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0
        
    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().astype(int)
        y_pred = y_pred.cpu().numpy() >= 0
        last_n = self.n
        self.n += len(y_true)
        true_count = np.sum(y_true == y_pred)
        # incremental update
        self.avg = true_count / self.n + last_n / self.n * self.avg

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion, 
        loss_meter, 
        score_meter
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        self.loss_meter = loss_meter
        self.score_meter = score_meter
        
        self.best_valid_score = -np.inf
        self.n_patience = 0
        
        self.messages = {
            "epoch": "[Epoch {}: {}] loss: {:.5f}, score: {:.5f}, time: {} s",
            "checkpoint": "The score improved from {:.5f} to {:.5f}. Save model to '{}'",
            "patience": "\nValid score didn't improve last {} epochs."
        }
    
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):        
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_loss, train_score, train_time = self.train_epoch(train_loader)
            valid_loss, valid_score, valid_time = self.valid_epoch(valid_loader)
            
            self.info_message(
                self.messages["epoch"], "Train", n_epoch, train_loss, train_score, train_time
            )
            
            self.info_message(
                self.messages["epoch"], "Valid", n_epoch, valid_loss, valid_score, valid_time
            )

            if True:
#             if self.best_valid_score < valid_score:
                self.info_message(
                    self.messages["checkpoint"], self.best_valid_score, valid_score, save_path
                )
                self.best_valid_score = valid_score
                self.save_model(n_epoch, save_path)
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message(self.messages["patience"], patience)
                break
            
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        train_loss = self.loss_meter()
        train_score = self.score_meter()
        
        for step, batch in enumerate(train_loader, 1):
            X = batch["X"].to(self.device)
            targets = batch["y"].to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(X).squeeze(1)
            
            loss = self.criterion(outputs, targets)
            loss.backward()

            train_loss.update(loss.detach().item())
            train_score.update(targets, outputs.detach())

            self.optimizer.step()
            
            _loss, _score = train_loss.avg, train_score.avg
            message = 'Train Step {}/{}, train_loss: {:.5f}, train_score: {:.5f}'
            self.info_message(message, step, len(train_loader), _loss, _score, end="\r")
        
        return train_loss.avg, train_score.avg, int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        valid_loss = self.loss_meter()
        valid_score = self.score_meter()

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                X = batch["X"].to(self.device)
                targets = batch["y"].to(self.device)

                outputs = self.model(X).squeeze(1)
                loss = self.criterion(outputs, targets)

                valid_loss.update(loss.detach().item())
                valid_score.update(targets, outputs)
                
            _loss, _score = valid_loss.avg, valid_score.avg
            message = 'Valid Step {}/{}, valid_loss: {:.5f}, valid_score: {:.5f}'
            self.info_message(message, step, len(valid_loader), _loss, _score, end="\r")
        
        return valid_loss.avg, valid_score.avg, int(time.time() - t)
    
    def save_model(self, n_epoch, save_path):
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            save_path,
        )
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


skf = StratifiedKFold(n_splits=2, random_state=42, shuffle=True)
for fold, (train_index, valid_index) in enumerate(skf.split(train_df, train_df["target"])):
    train_X = train_df.iloc[train_index]
    valid_X = train_df.iloc[valid_index][:20000] # Reduce calculation time
    print(train_X.shape, valid_X.shape)

    train_data_retriever = DataRetriever(
        train_X["id"].values, 
        train_X["target"].values, 
    )

    valid_data_retriever = DataRetriever(
        valid_X["id"].values, 
        valid_X["target"].values,
    )

    train_loader = torch_data.DataLoader(
        train_data_retriever,
        batch_size=32,
        shuffle=True,
        num_workers=8,
    )

    valid_loader = torch_data.DataLoader(
        valid_data_retriever, 
        batch_size=32,
        shuffle=False,
        num_workers=8,
    )

    model = Model()
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch_functional.binary_cross_entropy_with_logits

    trainer = Trainer(
        model, 
        device, 
        optimizer, 
        criterion, 
        LossMeter, 
        AccMeter
    )

    history = trainer.fit(
        1, 
        train_loader, 
        valid_loader, 
        f"best-model-{fold}.pth", 
        100,
    )

In [ ]:
models = []
for i in range(2):
    model = Model()
    model.to(device)
    
    checkpoint = torch.load(f"best-model-{i}.pth")
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    models.append(model)

In [ ]:
class DataRetriever(torch_data.Dataset):
    def __init__(self, paths):
        self.paths = paths

        self.q_transform = CQT1992v2(
            sr=2048, fmin=20, fmax=1024, hop_length=32
        )
          
    def __len__(self):
        return len(self.paths)
    
    def __get_qtransform(self, x):
        image = []
        for i in range(3):
            waves = x[i] / np.max(x[i])
            waves = torch.from_numpy(waves).float()
            channel = self.q_transform(waves).squeeze().numpy()
            image.append(channel)
            
        return torch.tensor(image).float()
    
    def __getitem__(self, index):
        file_path = convert_image_id_2_path(self.paths[index], is_train=False)
        x = np.load(file_path)
        image = self.__get_qtransform(x)
            
        return {"X": image, "id": self.paths[index]}

In [ ]:
test_data_retriever = DataRetriever(
    submission["id"].values, 
)

test_loader = torch_data.DataLoader(
    test_data_retriever,
    batch_size=32,
    shuffle=False,
    num_workers=8,
)

In [ ]:
y_pred = []
ids = []

for e, batch in enumerate(test_loader):
    print(f"{e}/{len(test_loader)}", end="\r")
    with torch.no_grad():
        tmp_pred = np.zeros((batch["X"].shape[0], ))
        for model in models:
            tmp_res = torch.sigmoid(model(batch["X"].to(device))).cpu().numpy().squeeze()
            tmp_pred += tmp_res / 2
        y_pred.extend(tmp_pred)
        ids.extend(batch["id"])

In [ ]:
submission = pd.DataFrame({"id": ids, "target": y_pred})
submission.to_csv("model_submission.csv", index=False)

In [ ]:
submission

https://github.com/JonasHeinzmann-AI/G2Net-Gravitational-Wave-Detection/blob/main/g2net-pytorch-ext.ipynb

In [ ]:
#This is simply pass data that Q-transformed with the size (x,y,3) to an EfficientNet, 
#the fully connected layer near the end of EfficientNet replaced by another fully connected 
#layer that serve our purpose.

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = efficientnet_pytorch.EfficientNet.from_pretrained("efficientnet-b7")
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
    
    def forward(self, x):
        out = self.net(x)
        return out

In [ ]:
class LossMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0

    def update(self, val):
        self.n += 1
        # incremental update
        self.avg = val / self.n + (self.n - 1) / self.n * self.avg

        
class AccMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0
        
    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().astype(int)
        y_pred = y_pred.cpu().numpy() >= 0
        last_n = self.n
        self.n += len(y_true)
        true_count = np.sum(y_true == y_pred)
        # incremental update
        self.avg = true_count / self.n + last_n / self.n * self.avg

In [ ]:
def main():
    device = torch.device("cuda")
    torch.cuda.synchronize

    model = Model()
    model = nn.DataParallel(model)
    model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
    criterion = torch_functional.binary_cross_entropy_with_logits

    trainer = Trainer(
        model,
        device,
        optimizer,
        criterion,
        LossMeter,
        AccMeter
    )

    history = trainer.fit(
        2,
        train_loader,
        valid_loader,
        "best-model.pth",
        100,
    )

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
# checkpoint = torch.load("best-model.pth")

# model.load_state_dict(checkpoint["model_state_dict"])
# model.eval();

https://github.com/SiddharthPatel45/gravitational-wave-detection/blob/main/code/gw-detection-modelling.ipynb

baseline score

In [ ]:
# Target distribution in the training data
train['target'].value_counts(normalize=True)

In [ ]:
# let's define some signal parameters
sample_rate = 2048 # data is provided at 2048 Hz
signal_length = 2 # each signal lasts 2 s
fmin, fmax = 20, 1024 # filter above 20 Hz, and max 1024 Hz (Nyquist freq = sample_rate/2)
hop_length = 64 # hop length parameter for the stft

# model compile params
batch_size = 250 # size in which data is processed and trained at-once in model
epochs = 3 # number of epochs (keep low as dataset is quite large 3~5 is enough as observed)

tf data pipeline ( preprocessing)

In [ ]:
# function to return the npy file corresponding to the id
def get_npy_filepath(id_, is_train=True):
    path = ''
    if is_train:
        return f'../input/g2net-gravitational-wave-detection/train/{id_[0]}/{id_[1]}/{id_[2]}/{id_}.npy'
    else:
        return f'../input/g2net-gravitational-wave-detection/test/{id_[0]}/{id_[1]}/{id_[2]}/{id_}.npy'

In [ ]:
# Define the Constant Q-Transform
cq_transform = CQT1992v2(sr=sample_rate, fmin=fmin, fmax=fmax, hop_length=hop_length)

# check if GPU enabled, then run the transform on GPU for faster execution
# if tf.test.is_gpu_available():
#     cq_transform = cq_transform.to('cuda')

In [ ]:
# function to load the file, preprocess, return the respective Constant Q-transform
def parse_function(id_path):
    # load the npy file
    signals = np.load(id_path.numpy())
    
    # loop through each signal
    for i in range(signals.shape[0]):
        # normalize the signal data
        signals[i] /= np.max(signals[i])
    
    # stack the arrays into a single vector
    signals = np.hstack(signals)
    
    # convert the signals to torch.tensor to pass to CQT
    signals = torch.from_numpy(signals).float()
    
    # get the CQT
    image = cq_transform(signals)
    
    # conver the image from torch.tensor to array
    image = np.array(image)
    
    # transpose the image to get right orientation
    image = np.transpose(image,(1,2,0))
    
    # conver the image to tf.tensor and return
    return tf.convert_to_tensor(image)

In [ ]:
# plot a sample
image = parse_function(tf.convert_to_tensor(get_npy_filepath(train['id'].values[0])))
print(image.shape)
plt.imshow(image)

In [ ]:

# From the Constant Q-Transform that we got, get the shape
input_shape = (69, 193, 1)

In [ ]:
# Get the feature ids and target
X = train[['id']]
y = train['target'].astype('int8').values

In [ ]:
# Split the training IDs into training & validation datasets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, stratify=y)

# Assign the test IDs
X_test = sample_sub[['id']]

In [ ]:
# the tf_function which is called in the data pipeline. This runs as TF function
def tf_parse_function(id_path, y=None):
    # pass the id_path to the py_function parse_function
    [x] = tf.py_function(func=parse_function, inp=[id_path], Tout=[tf.float32])
    
#     x.set_shape(signal_shape) # signal_shape
    x = tf.ensure_shape(x, input_shape)
    
    # if train/valid then return x, y; for test only return x
    if y is None:
        return x
    else:
        return x, y

In [ ]:
# train dataset
# Get the data filepaths as tensor_slices
train_dataset = tf.data.Dataset.from_tensor_slices((X_train['id'].apply(get_npy_filepath).values, y_train))

# shuffle the dataset
train_dataset = train_dataset.shuffle(len(X_train))

# apply the map method to tf_parse_function()
train_dataset = train_dataset.map(tf_parse_function, num_parallel_calls=tf.data.AUTOTUNE)

# set batch size of the dataset
train_dataset = train_dataset.batch(batch_size)

# prefetch the data
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
# valid dataset
# Get the data filepaths as tensor_slices
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid['id'].apply(get_npy_filepath).values, y_valid))

# apply the map method to tf_parse_function()
valid_dataset = valid_dataset.map(tf_parse_function, num_parallel_calls=tf.data.AUTOTUNE)

# set batch size of the dataset
valid_dataset = valid_dataset.batch(batch_size)

# prefetch the data
valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
# test dataset
# Get the data filepaths as tensor_slices
test_dataset = tf.data.Dataset.from_tensor_slices((X_test['id'].apply(get_npy_filepath, is_train=False).values))

# apply the map method to tf_parse_function()
test_dataset = test_dataset.map(tf_parse_function, num_parallel_calls=tf.data.AUTOTUNE)

# set batch size of the dataset
test_dataset = test_dataset.batch(batch_size)

# prefetch the data
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

modelling

In [ ]:
# Get the signal input shape
for x, _ in train_dataset.take(1):
    input_shape = x.shape[1:]

In [ ]:
# function to print the learning curves for the models
# adapted from https://www.tensorflow.org/tutorials/images/transfer_learning#learning_curves
def learning_curves(history, model='cnn'):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    if model == 'efn':
        auc = history.history['auc_2']
        val_auc = history.history['val_auc_2']
    else:
        auc = history.history['auc']
        val_auc = history.history['val_auc']
    
    plt.figure(figsize=(12, 12))    
    plt.subplot(3, 1, 1)
    plt.plot(auc, label='Training AUC')
    plt.plot(val_auc, label='Validation AUC')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,1.0])
    plt.title('Training and Validation AUC')
    plt.xlabel('epoch')
    plt.show()
    
    plt.subplot(3, 1, 2)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')
    plt.show()

    plt.subplot(3, 1, 3)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
# Instantiate the Sequential model
model_cnn = Sequential(name='CNN_model')

# Add the first Convoluted2D layer w/ input_shape & MaxPooling2D layer followed by that
model_cnn.add(Conv2D(filters=16,
                     kernel_size=3,
                     input_shape=input_shape,
                     activation='relu',
                     name='Conv_01'))
model_cnn.add(MaxPooling2D(pool_size=2, name='Pool_01'))

# Second pair of Conv1D and MaxPooling1D layers
model_cnn.add(Conv2D(filters=32,
                     kernel_size=3,
                     input_shape=input_shape,
                     activation='relu',
                     name='Conv_02'))
model_cnn.add(MaxPooling2D(pool_size=2, name='Pool_02'))

# Third pair of Conv1D and MaxPooling1D layers
model_cnn.add(Conv2D(filters=64,
                     kernel_size=3,
                     input_shape=input_shape,
                     activation='relu',
                     name='Conv_03'))
model_cnn.add(MaxPooling2D(pool_size=2, name='Pool_03'))

# Add the Flatten layer
model_cnn.add(Flatten(name='Flatten'))

# Add the Dense layers
model_cnn.add(Dense(units=512,
                activation='relu',
                name='Dense_01'))
model_cnn.add(Dense(units=64,
                activation='relu',
                name='Dense_02'))

# Add the final Output layer
model_cnn.add(Dense(1, activation='sigmoid', name='Output'))

In [ ]:
# Display the CNN model architecture
model_cnn.summary()

In [ ]:
# compile the model with following parameters
# Optimizer: Adam (learning_rate=0.0001)
# loss: binary_crossentropy
# metrics: accuracy/AUC
model_cnn.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=[[AUC(), 'accuracy']])

In [ ]:
# Fit the data
history_cnn = model_cnn.fit(x=train_dataset,
                            epochs=epochs,
                            validation_data=valid_dataset,
                            batch_size=batch_size,
                            verbose=1)

In [ ]:
# save the model
model_cnn.save('./model_CNN.h5')